In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from src.load_data import *
from src.img_processing import *
from src.convert import *
from src.img_processing import process_folder_for_cropping, check_folder_cropping_quality

In [ ]:
train_path = "data/train"
test_path = "data/test"
val_path = "data/val"
images, labels = load_data(test_path)

print("Nombre total d'images chargées :", len(images))
# print("Nombre total de labels chargés :", len(labels))

In [ ]:
load_data(test_path)
plt.imshow(images[12])
plt.title(f"label : {labels[12]}")

print("Nombre total d'images chargées :", len(images))
# print("Nombre total de labels chargés :", len(labels))

In [ ]:
height = 256
width = 256

yes_folder_path = 'brain_tumor_dataset/yes/'
yes_output_folder = 'img_cropped/yes/'
process_folder_for_cropping(yes_folder_path, yes_output_folder, width, height)

no_folder_path = 'brain_tumor_dataset/no/'
no_output_folder = 'img_cropped/no/'
process_folder_for_cropping(no_folder_path, no_output_folder, width, height)

In [ ]:
test_path_clean = process_folder_for_cropping()
train_path_clean = process_folder_for_cropping()
val_path_clean = process_folder_for_cropping()
# plt.imshow(test)

In [ ]:
propre_yes = check_folder_cropping_quality(yes_output_folder)

In [ ]:
propre_no = check_folder_cropping_quality(no_output_folder)

In [ ]:
type(propre_yes)

In [ ]:
# Charger les données de test
test_images, test_labels = load_data(test_path)

In [ ]:
# # Créer un modèle VGG-16 pré-entraîné (ne pas inclure la couche dense finale)
# base_model = VGG16(include_top=False, input_shape=(224, 224, 3))

# NUM_CLASSES = 1

# model = Sequential()
# model.add(base_model)
# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(NUM_CLASSES, activation='sigmoid'))

# # Figer les poids du VGG
# model.layers[0].trainable = False

# # Compiler le modèle
# model.compile(
#     loss='binary_crossentropy',
#     optimizer=RMSprop(lr=1e-4),
#     metrics=['accuracy']
# )

# # Afficher la structure du modèle
# model.summary()

# # Créer un générateur d'images pour la data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     rescale=1./255,
#     shear_range=0.05,
#     brightness_range=[0.1, 1.5],
#     horizontal_flip=True,
#     vertical_flip=True
# )

# # Ajuster le générateur aux données d'entraînement
# datagen.fit(images)

# # Entraîner le modèle avec l'augmentation de données
# model.fit(datagen.flow(images, labels, batch_size=32),
#           epochs=10,
#           steps_per_epoch=len(images) // 32,
#           validation_data=(test_images, test_labels))

# # Évaluer le modèle sur les données de test
# test_loss, test_accuracy = model.evaluate(test_images, test_labels)
# print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')
